In [43]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
symbol = "TSLA"
url = "https://data.alpaca.markets/v2/stocks/bars?symbols="+symbol+"&timeframe=5min&start=2025-03-20T01%3A02%3A03.123456789Z&limit=10000&adjustment=raw&feed=boats&sort=asc"

load_dotenv()

# Access the environment variables
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": api_key,
    "APCA-API-SECRET-KEY": api_secret,
}

response = requests.get(url, headers=headers)

data = response.json()
df_night = pd.DataFrame(data["bars"][symbol])


In [44]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
url = "https://data.alpaca.markets/v2/stocks/bars?symbols="+symbol+"&timeframe=5min&start=2025-03-20T01%3A02%3A03.123456789Z&limit=10000&adjustment=raw&feed=sip&sort=asc"

load_dotenv()

# Access the environment variables
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": api_key,
    "APCA-API-SECRET-KEY": api_secret,
}

response = requests.get(url, headers=headers)

data = response.json()
df_day = pd.DataFrame(data["bars"][symbol])


In [45]:
df = pd.concat([df_day, df_night], ignore_index=True)


In [46]:
import plotly.graph_objects as go

df['t'] = pd.to_datetime(df['t'])

# Generate vertical lines for each day at 13:30 and 20:00 UTC
dates = df['t'].dt.date.unique()
vlines = []

for date in dates:
    for hour, minute in [(8, 0),(13, 30), (20, 0)]:
        ts = pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour, minute=minute, tz='UTC')
        vlines.append(dict(
            type="line",
            x0=ts, x1=ts,
            y0=df['l'].min(), y1=df['h'].max(),
            line=dict(color="blue", width=1, dash="dot"),
            layer="below"
        ))

# Create figure
fig = go.Figure(data=[
    go.Candlestick(
        x=df['t'],
        open=df['o'],
        high=df['h'],
        low=df['l'],
        close=df['c'],
        increasing_line_color='green',
        decreasing_line_color='red'
    )
])

# Add vertical lines
fig.update_layout(
    title='Candlestick Chart with 13:30 and 20:00 UTC Lines',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    shapes=vlines
)

fig.show()

In [ ]:
#Study evolution during pre market / after hours and gap
# Overnight 00:00 -> 8:00 (boats)
# pre market 8:00 -> 13:30
# open 13:30 -> 20:00
# after hours 20:00 -> 00:00

# Pour chaque type : 
# gap weekend 
# gap open/close market 
# gap open/close pre market / after hours 
# Random walk tester